# Laboratorio 2 - DataFrames y Spark SQL (PySpark)

En este notebook trabajaremos el mismo caso retail (`transacciones_retail_large.csv`) pero usando:
- **DataFrames** (API declarativa, optimizada por Catalyst/Tungsten).
- **Spark SQL** (consultas SQL sobre vistas temporales).

Objetivo: calcular **ventas totales por tienda** y extender el análisis con consultas adicionales.


## 0. Preparación (Init): SparkSession

Para DataFrames y Spark SQL, el punto de entrada recomendado es `SparkSession`.


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Retail-DataFrames-SQL").getOrCreate()
spark

## 1. Carga del CSV con esquema

Definimos un **schema explícito** para evitar inferencias lentas o errores de tipos.


In [ ]:
# Basado en Init_DataFrame_PySpark.py
from pyspark.sql.types import (
    StructType, StructField,
    StringType, IntegerType, DoubleType, TimestampType
)

schema_transacciones = StructType([
    StructField("transaccion_id", StringType(), False),
    StructField("timestamp", TimestampType(), True),
    StructField("tienda_id", StringType(), True),
    StructField("producto_id", StringType(), True),
    StructField("categoria", StringType(), True),
    StructField("cantidad", IntegerType(), True),
    StructField("precio_unitario", DoubleType(), True),
    StructField("metodo_pago", StringType(), True),
])

df_ventas = spark.read.csv(
    "transacciones_retail_large.csv",
    header=True,
    schema=schema_transacciones
)

df_ventas.printSchema()
df_ventas.show(5, truncate=False)

## 2. Transformaciones con DataFrames

Calculamos el ingreso por fila (`cantidad * precio_unitario`) y luego agregamos por tienda.


In [ ]:
from pyspark.sql.functions import col, sum as spark_sum, desc

df_calculado = df_ventas.withColumn(
    "ingreso_venta",
    col("cantidad") * col("precio_unitario")
)

df_reporte = (df_calculado
              .groupBy("tienda_id")
              .agg(spark_sum("ingreso_venta").alias("venta_total"))
              .orderBy(desc("venta_total"))
             )

df_reporte.show(10, truncate=False)

## 3. El puente a SQL: vista temporal

Registramos el DataFrame como una **vista temporal** para consultarla con SQL.


In [ ]:
# Basado en Init_SparkQL.py
df_ventas.createOrReplaceTempView("v_transacciones")
spark.catalog.listTables()

## 4. Consulta Spark SQL

Escribimos SQL estándar para obtener el total de ventas por tienda.


In [ ]:
query = """
SELECT
    tienda_id,
    SUM(cantidad * precio_unitario) AS venta_total
FROM v_transacciones
GROUP BY tienda_id
ORDER BY venta_total DESC
"""

df_resultado_sql = spark.sql(query)
df_resultado_sql.show(10, truncate=False)

## 5. Extensiones útiles (opcional)

A continuación, algunas consultas típicas para extender el análisis.


In [ ]:
# 5.1 Ventas por categoría
spark.sql("""
SELECT
  categoria,
  SUM(cantidad * precio_unitario) AS venta_total
FROM v_transacciones
GROUP BY categoria
ORDER BY venta_total DESC
""").show(10, truncate=False)

In [ ]:
# 5.2 Ventas por método de pago
spark.sql("""
SELECT
  metodo_pago,
  SUM(cantidad * precio_unitario) AS venta_total
FROM v_transacciones
GROUP BY metodo_pago
ORDER BY venta_total DESC
""").show(truncate=False)

In [ ]:
# 5.3 Ticket promedio por tienda (venta_total / número de transacciones)
spark.sql("""
SELECT
  tienda_id,
  SUM(cantidad * precio_unitario) AS venta_total,
  COUNT(*) AS n_transacciones,
  (SUM(cantidad * precio_unitario) / COUNT(*)) AS ticket_promedio
FROM v_transacciones
GROUP BY tienda_id
ORDER BY ticket_promedio DESC
""").show(10, truncate=False)

## 6. Ejercicios sugeridos

1. Ventas por día (si `timestamp` está completo) usando `DATE(timestamp)`.
2. Top 10 productos por venta total.
3. ¿Cuál es la distribución de `cantidad` por categoría?
4. Compara el plan con `df_resultado_sql.explain(True)` y `df_reporte.explain(True)`.


## 7. Cierre

Cuando termines, puedes detener la sesión:


In [ ]:
# spark.stop()